# Downstream Analysis of Single-Cell Data  -- <font color='teal'>Tutorial Pipeline</font>

<br><font color='red'>**Warning 1:** These materials were put together relatively quickly and have not been thoroughly double-checked and optimized yet! Any feedback that may help to improve these materials is very welcome! </font><br><br>

<font color='red'>**Warning 2:** These materials use an example image with a quite uniform population of cells (meaning the cells are in general quite similar). As a consequence, there aren't really any interesting patterns that can be uncovered by data analysis, which is a bit unfortunate for a tutorial on this topic. However, the concepts can still be explained - it's just that the outcome won't be too exciting. We are planning to switch to more interesting example images in the future.</font>

## Introductory Notes


### About this Tutorial

This tutorial provides an introductory overview of how to approach single-cell analysis following image segmentation.  

For people with limited experience in data analysis, this tutorial is intended as an inspiration and incentive to think about possible advanced analyses downstream of segmentation. Solving the exercises without help may be difficult, so it may be a good idea to have a look at the solutions to get some idea of how the problems should be approached. However, once the principles are understood, it is an important part of the learning experience to build your own implementation and to play around with different possibilities.

People more experienced in data science can use this tutorial as a starting point for exploring the data analysis packages available in python. It also illustrates that python readily allows the construction of complete and consistent analysis pipelines, from image preprocessing to feature extraction to clustering. 


### Concepts Discussed in this Tutorial

- Feature extraction
    - Specific features of interest
    - General image descriptors


- Feature space standardization


- Dimensionality reduction
    - PCA
    - tSNE


- Clustering by k-means (including Elbow plots)


- Graph/network-based data representation and visualization



### Data Analysis with Python

There are a number of data analysis, machine learning, clustering and other data analysis packages for Python. The following is a list of some commonly used packages:

- [Pandas](http://pandas.pydata.org/)
    - Provides a dataframe object ideal for handling `sample x feature` data
    - Most of the packages mentioned below can seamlessly work with pandas dataframes
    - *Note:* the solutions to this tutorial currently do *not* use pandas *but will be updated to do so in the future!*


- [scikit-learn](http://scikit-learn.org/stable/)
    - Large data analysis and machine learning package featuring many standard and state-of-the-art algorithms


- [scipy.cluster](http://docs.scipy.org/doc/scipy/reference/cluster.html)
    - Scipy implementation of clustering algorithms


- [networkx](http://networkx.github.io/)
    - Package for graph-based analysis and visualization of data


- [keras](https://keras.io/)
    - Extensive state-of-the-art package for deep learning
    - Provides APIs for Theano and TensorFlow
    - *Note:* this tutorial currently does *not* cover deep learning
    

- [pyMC](https://pymc-devs.github.io/pymc/)
    - Package for Bayesian modeling and Markov-Chain Monte-Carlo sampling
    - *Note:* this tutorial currently does *not* cover Bayesian inference

Most of these packages use numpy arrays (or pandas dataframes) to handle data, so being familiar with numpy is a prerequisite to working with any of the above. If you have completed the main tutorial, you should not have any major problems with this.


### Setup

The following packages/modules need to be installed for this tutorial. With the exception of `tifffile`, they should all come pre-installed with the Anaconda distribution or sould be easy to install using `conda install` or `pip install`.

- **Already needed for main tutorial**
    - python 2.7 *(or python 3.x, if you don't mind adjusting the solutions)*
    - numpy
    - scikit-image
    - matplotlib
    - scipy
    - tifffile


- **New for this tutorial**
    - scikit-learn 
    - networkx

Make sure these modules are installed before you proceed.

Note that this tutorial is based on the segmentation and extracted measures from `example_image_1.tif` from the main pipeline. The necessary files to run the solutions are included in this directory, but it might be a bit more interesting for you to use the results of your own work.

## Preparation

In [ ]:
### Importing modules

from __future__ import division    # Python 2.7 legacy
import numpy as np                 # Array manipulation package
import matplotlib.pyplot as plt    # Plotting package
import scipy.ndimage as ndi        # Multidimensional image operations

import sklearn as skl              # Data analysis and machine learning
import networkx as nx              # Graphs/networks

import json                        # File handling

### <font color='teal'>Exercise 0</font>

Reload the image, the segmentation, and the results dictionary from the main tutorial. The labeled cell edges may also be useful.

In [ ]:
### Loading image and segmentation data from the main tutorial

# YOUR CODE HERE!

## Feature Extraction

As discussed in the main tutorial, we can measure various quantities for each cell individually (once the cells have been segmented). These quantities can be considered 'features' for the purpose of further analysis such as clustering. Besides explicitly measured specific quantities, there are also algorithms that automatically extract a whole bunch of features from an image.

All the extracted features together are called the 'feature space'. Each sample can be considered a point in feature space, which has as many dimensions as there are features. The feature space should be arranged as an array of shape `(n_samples,n_features)` or in a pandas `df`.

### <font color='teal'>Exercise 1</font>

You should already have a set of single-cell measurements ('features') from the main tutorials, stored in the results dict.

Try to think of a few (at least 2) additional features and extract them from the image/segmentation. Be creative and try to find features that might contain biologically interesting information!

Combine all the features in a 'feature_space' array of shape `(n_samples,n_features)`, were `n_samples` in our case is the number of cells. You could also use a pandas dataframe for this. Be sure to use a data type that works for all the different types of features you are using. If you are using numpy arrays, it makes sense to also keep a list of column labels (names of the features) and row labels (cell IDs) because the array itself is unlabeled.

- **Hint 1:** For many measures of shape and spatial distribution, it is useful to first calculate the `centroid` of the segmented object and then think of features relative to it.


- **Hint 2:** It can be advantageous to use measures that are largely independent of cell size (or normalized for cell size) to prevent cell size from dominating the different features. Of course, cell size itself should be a feature.


- **Hint 3:** Don't forget that we also identified the boundaries of each cell in the main script. Importing or reconstructing this date here may be useful for the calculation of various features.


- **Hint 4:** Make sure you visualize your data!
    - It can be very useful to have a look at what a feature looks like when mapped to the actual image (using a semi-transparent overlay, see section 'Expansion by Watershed' in the main tutorial). 
    - This may already show interesting patterns, or should at least confirm that the extracted values are consistent with the feature's rationale.
    - Also, box and scatter plots are great options for checking how the values of a feature are distributed and how features relate to each other.

In [ ]:
### Initialize feature space array and add previously extracted features

# YOUR CODE HERE!

In [ ]:
### Get additional features and add them to the feature space

# YOUR CODE HERE!

### <font color='teal'>Exercise 2</font>

Use a feature extraction algorithm that returns a large feature set for each cell; an image **descriptor**. The features could for example be related to shape or texture. Feel free to search around the internet for a bit to see what kind of interesting algorithms are available. The algorithm should produce a second feature space that once again has the shape `(n_samples,n_features)`.

The one used in the solutions is `skimage.feature.daisy`, an algorithm for the extraction of local image features based on a grayscale image. It should be pointed out, however, that membrane images are not what [DAISY](scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.daisy) would typically be applied to; it is best used for images with more local patterning/texture.

In [ ]:
### Extract descriptor features

# YOUR CODE HERE!

### <font color='teal'>Exercise 3</font>

Just as with the measurements in the main tutorial, it can be interesting to check how the extracted features map onto the actual cells. You can do this with an appropriately colored semi-transparent overlay, just as in the main tutorial.

Have a look at your newly constructed features to see if they make sense. You can also have a look at some of the DAISY features.

Besides the semi-transparent overlays, the population data we now have available can of course also be visualized and investigated by means of scatter plots and boxplots. Feel free to play around a bit!

In [ ]:
### Visualize the extracted features

# YOUR CODE HERE!

## Normalization and Standardization

Many classification and clustering algorithms need features to be normalized and/or standardized, otherwise the absolute value range of the feature could affect the result (for example, you could get a different result if you use cell size in um or in pixels, because the absolute numbers are different).

Normalization in this context generally means scaling your features a range from 0 to 1. Standardization means centering the features around zero and scaling them to "unit variance" by dividing them by their standard deviation (sometimes called "whitening").

It's worthwhile to read up on normalization/standardization so you avoid introducing errors/biases. For example, normalization of data with outliers will compress the 'real' data into a very small range. Thus, outliers should
be removed before normalization/standardization.

### <font color='teal'>Exercise 4</font>

In preparation for feature rescaling, find a sensible way to remove outliers from your feature space.

Be mindful of implicit assumptions and potential bias that might come with outlier removal!

In [ ]:
### Remove outliers

# YOUR CODE HERE!

### <font color='teal'>Exercise 5</font>

Standardize or normalize your feature space as you deem fit, either manually or using a module function.

Don't forget to visualize the result using a boxplot!

In [ ]:
# YOUR CODE HERE!

## Principal Component Analysis (PCA)

The principal components of a feature space are the axes of greatest variance
of the data. By transforming our data to this "relevance-corrected" coordinate
system, we can achieve two things:

1. Usually, most of the variance in a dataset falls onto just a few principal
   components, so we can ignore the other ones as irrelevant, thus reducing
   the number of features whilst maintaining all or most of the information 
   in the data ('dimensionality reduction'). For many downstream analyses,
   this can improve both the quality of the result and the computational performance.


2. Just PCA on its own can yield interesting results. For example, different cell
   populations that are not clearly separated by any single feature may 
   appear separated along a principal component ('clustering'). Furthermore, principal 
   components may correlate with features of the data, which helps us understand the impact
   these features have on downstream analyses. It can also help us understand how abstract
   features such as the DAISY descriptors relate to the image.

### <font color='teal'>Exercise 6</font>

Perform a PCA on your feature space and investigate the results.

You may want to use the PCA implementation of scikit-learn. Algorithms in sklearn are provided as "estimator objects". The general workflow for using them is as follows:

1. Instantiate the estimator object and pass it general parameters.
2. Fit the estimator to your data.
3. (Optional: extract information about your data from the estimator.)
4. Transform your data to the reference space of the estimator.
5. (Optional: discard some dimensions of the transformed data, e.g. the dimensions that explain very little of the populations variance in a PCA. This can also be done from the start, by telling the estimator to reduce the data to a set number of dimensions.)

Investigate the results of your PCA using scatter plots (and other visualizations if you like):

- Plot PCs against each other (PC1 vs PC2, ...)
- Plot PCs against features (PC1 vs feature1, feature 2, ...)

In [ ]:
### Performing PCA with scikit-learn

# YOUR CODE HERE!

In [ ]:
### Visualizing the outcome of the PCA

# YOUR CODE HERE!

## K-means clustering

If you expect that you can split your population into distinct groups, an easy way of doing so in an unsupervised fashion is k-means clustering. K-means partitions samples into clusters based on their proximity to the cluster's centroid.

### <font color='teal'>Exercise 7</font>

Perform k-means clustering on your data. You can use sklearn or scipy to do so.

To do so, you have to decide the number of clusters from the start. Just try it with 5 clusters to begin with. You can try clustering raw, normalized/standardized and/or PCA-transformed data to see if there is a difference.

Note that we can't really hope for a clear separation of our population into meaningful clusters because our population of cells is very uniform to begin with.

Visualize your results as color-coded scatterplots and as a semi-transparent map to the image.

In [ ]:
### K-means clustering using scipy

# YOUR CODE HERE

### <font color='teal'>Bonus Exercise</font>

Can you find/implement a way of objectively choosing the number of clusters for k-means?

In [ ]:
# YOUR CODE HERE!

## tSNE Analysis

Although PCA is great to reduce and visualize high-dimensional data, it only
works well on linear relationships and global trends. Therefore, alternative 
algorithms optimized for non-linear, local relationships have also been
created.

These algorithms tend to be quite complicated and going into them is beyond 
the scope of this tutorial. This example is intended as a taste of what is out
there and to show people who already know about these methods that they are
available in Python. Note that it can be risky to use these algorithms if 
you do not know what you are doing, so it may make sense to read up and/or to 
consult with an expert before you do this kind of analysis.

This example uses the tSNE implementation in scikit-learn. tSNE (or t-distributed
Stochastic Neighbor Embedding) is a machine learning algorithm that attempts to
project the high-dimensional feature data into just 2 or 3 dimensions, preserving
the proximity between samples as much as possible. In other words, samples that
are close together in the full nD feature space should end up close together in
the reduced 2D/3D feature space.

In [ ]:
### tSNE with scikit-learn

# YOUR CODE HERE or JUST CHECK OUT THE EXAMPLE IN THE SOLUTIONS!

## Graph-Based Analysis

Graphs are a universal way of mathematically describing relationships, be 
they based on similarity, interaction, or virtually anything else. Despite 
their power, graph-based analyses have so far not been used extensively on 
biological imaging data, but as microscopes and analysis algorithms improve,
they become increasingly feasible and will likely become very important in
the future.

The networkx module provides various functions for importing and generating
graphs, for operating on and analyzing graphs, and for exporting and visualizing
graphs. The following example shows how a simple graph based on our feature 
space could be constructed and visualized. In doing so, it introduces the networkx 
Graph object, which is the core of the networkx module.

For this example, I am using the feature space generated by the DAISY descriptor,
so I first need to clean it (remove outliers, standardize) and reduce its dimensionality, much
as we did for the other feature space above.

In [ ]:
### Graph construction and visualization with networkx

# YOUR CODE HERE or JUST CHECK OUT THE EXAMPLE IN THE SOLUTIONS!

### <font color='teal'>*Congratulations! You have completed the tutorial!*</font>

<br>

**...but if you now just go back to your work and do nothing, you will forget all you learned within a month or two!**


So, what to do?


- Start applying what you have learned to your own work!


- Stay engaged even if you currently don't need your new skills at work!

    - Play around with data from your work, even if you don't need it at the moment

    - Find yourself an interesting little 'pet project' to play around with

    - Look for tutorials online with additional/advanced content
    
    - Join for seminars/events related to coding and image analysis
        - Check out the [Bio-IT Portal](https://bio-it.embl.de/) for more info! *[internal access only]*
        - Join the [EMBL Coding Club](https://bio-it.embl.de/coding-club/) *[internal access only]*